# Efficient Yelp API Calls (Core)


## Assignment: 
- For this assignment, you will be working with the Yelp API.

- As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

- Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

- Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

## Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

In [4]:
with open('/Users/willrodemoyer/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [5]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
yelp = YelpAPI(login['api-key'], timeout_s= 5.0)

## Define Search

In [7]:
location= 'Denver, CO 80205'
term= 'pizza'

In [8]:
location.split(',')[0]

'Denver'

In [10]:
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok= True)

JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [11]:
JSON_FILE

'Data/Denver-pizza.json'

In [12]:
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Denver-pizza.json not found. Saving empty list to file.


In [13]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
# set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [15]:
results = yelp.search_query(location= location,
                                term=term)                        
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
type(results)

dict

In [17]:
len(results)

3

In [18]:
results['total']

675

In [19]:
results['businesses']

[{'id': 'Yk84dALUw0TPPf9sMuu2hg',
  'alias': 'redeemer-pizza-denver',
  'name': 'Redeemer Pizza',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/tmZdbkbfJ_qDmYzf6j8_0A/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/redeemer-pizza-denver?adjust_creative=nhyOXwry0_tSX8AWBmAcIw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nhyOXwry0_tSX8AWBmAcIw',
  'review_count': 128,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.5,
  'coordinates': {'latitude': 39.75975, 'longitude': -104.98405},
  'transactions': ['pickup', 'delivery'],
  'location': {'address1': '2705 Larimer St',
   'address2': None,
   'address3': '',
   'city': 'Denver',
   'zip_code': '80205',
   'country': 'US',
   'state': 'CO',
   'display_address': ['2705 Larimer St', 'Denver, CO 80205']},
  'phone': '+17207801379',
  'display_phone': '(720) 780-1379',
  'distance': 1321.4275024895007},
 {'id': 'B_1cqks5CI5N_viXXLuLWg',
  'alias': 'brooklyns-finest-piz

In [20]:
results['region']

{'center': {'longitude': -104.9688720703125, 'latitude': 39.76182745972278}}

In [25]:
results_per_page = len(results['businesses'])
results_per_page

20

In [26]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

34

In [28]:
for i in tqdm_notebook(range(1,n_pages+1)):
    try:
        time.sleep(.2) 
        
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        n_results = len(previous_results)
        
        results= yelp.search_query(location=location,
                                    term=term, 
                                    offset=n_results+1)
    
    
        
        previous_results.extend(results['businesses'])
    
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
        
    except Exception as e:
        print(' [!] ERROR', e)
    
    

  0%|          | 0/34 [00:00<?, ?it/s]

 [!] ERROR ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


In [29]:
df = pd.read_json(JSON_FILE)

In [30]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ofPa6i98oFCiq0zoWj1cvw,black-shirt-brewing-denver-5,Black Shirt Brewing,https://s3-media3.fl.yelpcdn.com/bphoto/meal5l...,False,https://www.yelp.com/biz/black-shirt-brewing-d...,411,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.5,"{'latitude': 39.7697672895761, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '3719 Walnut St', 'address2': '',...",+13039932799,(303) 993-2799,948.714510
1,n18mJPI6mQfBJEp8UrITyw,blue-pan-pizza-denver-4,Blue Pan Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/EAkVmo...,False,https://www.yelp.com/biz/blue-pan-pizza-denver...,697,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.735349, 'longitude': -104.945823}",[delivery],$$,"{'address1': '3509 E 12th Ave', 'address2': ''...",+17205190944,(720) 519-0944,3542.839351
2,B_1cqks5CI5N_viXXLuLWg,brooklyns-finest-pizza-denver-3,Brooklyn's Finest Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/KZf9Qb...,False,https://www.yelp.com/biz/brooklyns-finest-pizz...,215,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.746419, 'longitude': -104.993448}","[delivery, pickup]",$,"{'address1': '825 16th St Mall', 'address2': '...",+13035733779,(303) 573-3779,2710.962636
3,alVuEHMqyyTpXZKcIfKT9w,white-pie-denver,White Pie,https://s3-media3.fl.yelpcdn.com/bphoto/-v__Wc...,False,https://www.yelp.com/biz/white-pie-denver?adju...,411,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.74348039, 'longitude': -104.96...","[delivery, pickup]",$$,"{'address1': '1702 Humboldt St', 'address2': N...",+13038625323,(303) 862-5323,2040.644917
4,kI5oOBAGS-Vo4chwmgOVmQ,cart-driver-denver,Cart-Driver,https://s3-media1.fl.yelpcdn.com/bphoto/YzE2Pj...,False,https://www.yelp.com/biz/cart-driver-denver?ad...,641,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.7575443618608, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '2500 Larimer St', 'address2': 'S...",+13032923553,(303) 292-3553,1554.610229


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             660 non-null    object 
 1   alias          660 non-null    object 
 2   name           660 non-null    object 
 3   image_url      660 non-null    object 
 4   is_closed      660 non-null    bool   
 5   url            660 non-null    object 
 6   review_count   660 non-null    int64  
 7   categories     660 non-null    object 
 8   rating         660 non-null    float64
 9   coordinates    660 non-null    object 
 10  transactions   660 non-null    object 
 11  price          551 non-null    object 
 12  location       660 non-null    object 
 13  phone          660 non-null    object 
 14  display_phone  660 non-null    object 
 15  distance       660 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 78.1+ KB
